In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# !pip3 install --quiet scipy tenacity tiktoken termcolor openai --quiet

In [11]:
# Set up credentials:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4o-mini"
client = OpenAI(api_key=OPENAI_API_KEY)

In [12]:
# Example of a simple request:
completion = client.chat.completions.create(
  model=GPT_MODEL,
  messages=[
    {"role": "user", "content": "What's machine learning?"}
  ]
)
print(completion.choices[0].message.content)

Machine learning (ML) is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform tasks without explicit programming. Instead of being programmed with specific instructions, machine learning systems learn from data, identify patterns, and make decisions or predictions based on that data.

There are several key concepts in machine learning:

1. **Supervised Learning**: Involves training a model on a labeled dataset, where the input data is paired with the correct output. The model learns to map inputs to the desired output by minimizing the error between its predictions and the actual labels.

2. **Unsupervised Learning**: Involves training a model on data that does not have labeled responses. The system tries to identify patterns or group similar data points together without predefined classifications.

3. **Reinforcement Learning**: A type of learning where an agent learns to make decisions by taking

In [16]:
# Helper function

def chat_completion_request(client, messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def create_message(role, content):
    return {'role': role, 'content': content}

In [21]:
# Specify the tools:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [45]:
import json

def get_current_weather(location, format):
    return "Call successful from get_current_weather()."


def get_n_day_weather_forecast(location, format, num_days):
    return "Call successful from get_n_day_weather_forecast()"


def execute_function_call(message):
    args = json.loads(message.tool_calls[0].function.arguments)
    if message.tool_calls[0].function.name == "get_current_weather":
        results = get_current_weather(args["location"], args["format"])
    elif message.tool_calls[0].function.name == "get_n_day_weather_forecast":
        results = get_n_day_weather_forecast(args["location"], args["format"], args["num_days"])
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

In [17]:
# Define messages

messages = []
messages.append(
    create_message(
        "system",
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
)
messages.append(create_message("user", "What's the weather like today?"))

In [24]:
# Submit response

chat_response = chat_completion_request(
    client, messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide me with your city and state so I can check the current weather for you?', refusal=None, role='assistant', function_call=None, tool_calls=None))


In [27]:
# User replies with additional details:
messages.append(
    create_message(
        "user",
        "I'm in San Francisco, CA"
    )
)

In [30]:
# The Model responds:
chat_response = chat_completion_request(
    client, messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Do you prefer the temperature to be reported in Celsius or Fahrenheit?', refusal=None, role='assistant', function_call=None, tool_calls=None))


In [31]:
# User replies with clarification:
messages.append(
    create_message(
        "user",
        "In Fahrenheit please."
    )
)

In [32]:
# The Model responds:
chat_response = chat_completion_request(
    client, messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Rmep9YN7JcBxSNp1nWvVrLoN', function=Function(arguments='{"location":"San Francisco, CA","format":"fahrenheit"}', name='get_current_weather'), type='function')]))


In [48]:
latest_response = chat_response.choices[0]

if latest_response.finish_reason == "tool_calls":
    latest_msg = latest_response.message
    results = execute_function_call(latest_msg)
    messages.append({"role": "function",
                     "tool_call_id": latest_msg.tool_calls[0].id,
                     "name": latest_msg.tool_calls[0].function.name,
                     "content": results
                     })

In [49]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': "What's the weather like today?"},
 ChatCompletionMessage(content='Could you please provide me with your city and state so I can check the current weather for you?', refusal=None, role='assistant', function_call=None, tool_calls=None),
 {'role': 'user', 'content': "I'm in San Francisco, CA"},
 ChatCompletionMessage(content='Do you prefer the temperature to be reported in Celsius or Fahrenheit?', refusal=None, role='assistant', function_call=None, tool_calls=None),
 {'role': 'user', 'content': 'In Fahrenheit please.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Rmep9YN7JcBxSNp1nWvVrLoN', function=Function(arguments='{"location":"San Francisco, CA","format":"fahrenheit"}', name='get_current_weather'), type='fun